In [ ]:
import os
from pandas import read_table
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr, Metadata
from mutperiodpy.quick_scripts.GetNRL import getNRL
from mutperiodpy.GeneratePlotnineFigures import parseAndPlotPeriodicity, parseNucleosomeCountsDataForPlotting, plotPeriodicity

liDataDirectory = os.path.join(getDataDir(), "Li_tXR-seq")

In [ ]:
SINGLE_NUC = 1
NUC_GROUP = 2

liRawCounts = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
liSinglenucNormalizedCounts = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
liTrinucNormalizedCounts = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
liDamageNormalizedCounts = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
NRLs = dict()

nucleosomeMapNameWhitelist = ['hg19_hybrid_nucleosome_map', 'hg19_LCL_MNase_nuc_map_all_mappable', 'hg19_NHF1_MNase_nuc_map_all_mappable']
# nucleosomeMapNameWhitelist += ['hg19_LCL_MNase_nuc_map_all_mappable_euchromatin', 'hg19_LCL_MNase_nuc_map_all_mappable_heterochromatin',
#                               'hg19_LCL_MNase_nuc_map_all_mappable_stringent_euchromatin']

for nucleosomeCountsFilePath in getFilesInDirectory(liDataDirectory, DataTypeStr.rawNucCounts+".tsv"):

    countsMetadata = Metadata(nucleosomeCountsFilePath)
    if countsMetadata.nucPosName not in nucleosomeMapNameWhitelist: continue

    if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
    else: radius = SINGLE_NUC

    liRawCounts[radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

    if countsMetadata.nucPosName not in NRLs: NRLs[countsMetadata.nucPosName] = getNRL([countsMetadata.baseNucPosFilePath])[0]

for nucleosomeCountsFilePath in getFilesInDirectory(liDataDirectory, DataTypeStr.normNucCounts+".tsv"):

    countsMetadata = Metadata(nucleosomeCountsFilePath)
    if countsMetadata.nucPosName not in nucleosomeMapNameWhitelist: continue

    if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
    else: radius = SINGLE_NUC

    if "singlenuc" in os.path.basename(nucleosomeCountsFilePath):
        liSinglenucNormalizedCounts[radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)
    if "trinuc" in os.path.basename(nucleosomeCountsFilePath):
        liTrinucNormalizedCounts[radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)
    elif "custom_context" in os.path.basename(nucleosomeCountsFilePath):
        liDamageNormalizedCounts[radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

    if countsMetadata.nucPosName not in NRLs: NRLs[countsMetadata.nucPosName] = getNRL([countsMetadata.baseNucPosFilePath])[0]

In [ ]:
for radius in liRawCounts:
    for nucleosomeMapName in liRawCounts[radius]:
        plot = parseAndPlotPeriodicity(liRawCounts[radius][nucleosomeMapName], dataCol = "Aligned_Strands_Counts",
                                       smoothTranslational = False, nucRepLen = NRLs[nucleosomeMapName],
                                       title = f"Li {nucleosomeMapName} Raw", yAxisLabel = "Counts")
        print(plot)

In [ ]:
for radius in liSinglenucNormalizedCounts:
    for nucleosomeMapName in liSinglenucNormalizedCounts[radius]:
        plot = parseAndPlotPeriodicity(liSinglenucNormalizedCounts[radius][nucleosomeMapName], dataCol = "Normalized_Aligned_Strands",
                                       smoothTranslational = True, nucRepLen = NRLs[nucleosomeMapName],
                                       title = f"Li {nucleosomeMapName} Singlenuc Normalized", yAxisLabel = "Normalized Counts")
        print(plot)

In [ ]:
for radius in liTrinucNormalizedCounts:
    for nucleosomeMapName in liTrinucNormalizedCounts[radius]:
        plot = parseAndPlotPeriodicity(liTrinucNormalizedCounts[radius][nucleosomeMapName], dataCol = "Normalized_Aligned_Strands",
                                       smoothTranslational = True, nucRepLen = NRLs[nucleosomeMapName],
                                       title = f"Li {nucleosomeMapName} Trinuc Normalized", yAxisLabel = "Normalized Counts")
        print(plot)

In [ ]:
for radius in liDamageNormalizedCounts:
    for nucleosomeMapName in liDamageNormalizedCounts[radius]:
        plot = parseAndPlotPeriodicity(liDamageNormalizedCounts[radius][nucleosomeMapName], dataCol = "Normalized_Aligned_Strands",
                                       smoothTranslational = True, nucRepLen = NRLs[nucleosomeMapName],
                                       title = f"Li {nucleosomeMapName} Damage Normalized", yAxisLabel = "Normalized Counts")
        print(plot)